Useful imports

In [1]:
import heapq
import bisect
import collections
import itertools
import functools
import math

Assign cookies: https://leetcode.com/problems/assign-cookies/
Video Link: https://youtu.be/DIX2p7vb9co?si=-lYoEBkNX6G40Tpo

In [2]:
# https://leetcode.com/problems/assign-cookies/submissions/1280845661
def findContentChildren(g: list[int], s: list[int]) -> int:
    """
    For each child we assign cookies that only require as much to satisfy their greed.
    For a child with greed of 3, we assign the minimum cookie left to satisfy condition: s[k] >= 3
    """
    # Sort in ASC order since we need to find the max
    # number of content children possible
    g.sort()
    s.sort()

    # Have pointers for greed and cookies arrary
    g_length, s_length = len(g), len(s)
    content_children = g_idx = s_idx = 0

    # Iterate until either of the arrays are exhausted
    while g_idx < g_length and s_idx < s_length:
        if s[s_idx] >= g[g_idx]:
            content_children, s_idx, g_idx = content_children + 1, s_idx + 1, g_idx + 1
        else:
            s_idx += 1

    return content_children

# Testing the solution
assert findContentChildren([1,2,3], [1,1,1]) == 1
assert findContentChildren([1,2], [1,2,3]) == 2
assert findContentChildren([1,2,3], [1,2]) == 2

Shortest job first CPU scheduling
Video Link: https://youtu.be/3-QbX1iDbXs?si=oh3KUu3zLd08uDZ7

In [3]:
def SJF(bt: list[int]) -> float:
    """Time: O(N), Space: O(N)"""

    # Heapify to convert into a min heap
    heapq.heapify(bt)

    N = len(bt)
    curr_time = total_time = 0
    while bt:
        runtime = heapq.heappop(bt)
        total_time += curr_time
        curr_time += runtime

    avg_wait = total_time // N
    return avg_wait

# Testing the solution
assert SJF([4,3,7,1,2]) == 4
assert SJF([1,2,3,4]) == 2

Lemonade change: https://leetcode.com/problems/lemonade-change/description/
Video link: https://youtu.be/n_tmibEhO6Q?si=bqyD-xTjBPIG9scO

In [4]:
# https://leetcode.com/problems/lemonade-change/submissions/1281069969
def lemonadeChange(bills: list[int]) -> bool:
    change5 = change10 = 0
    for bill in bills:
        if bill == 5:
            change5 += 1
        elif bill == 10:
            change10 += 1
            change5 -= 1
        else:
            # Try to change with 10 if available
            if change10 > 0:
                change5, change10 = change5 - 1, change10 - 1
            # Else we can change with 5
            else:
                change5 -= 3

        # Reason this is added here and not at the end: [10,5]
        if change5 < 0 or change10 < 0:
            return False

    else:
        return True

# Testing the solution
assert lemonadeChange([5,5,10,10,20]) == False
assert lemonadeChange([10,5]) == False
assert lemonadeChange([5,10,20]) == False
assert lemonadeChange([5,5,5,20]) == True

Jump game: https://leetcode.com/problems/jump-game/
Video Link: https://youtu.be/tZAa_jJ3SwQ?si=PUeiia1qBXmvfk_C

In [5]:
def canJumpBrute(nums: list[int]) -> bool:
    N = len(nums)
    good_indices: collections.deque = collections.deque([N - 1])
    for i in range(N - 2, -1, -1):
        for good_idx in good_indices:
            if i + nums[i] >= good_idx:
                good_indices.appendleft(i)
                break

    return good_indices[0] == 0

# Testing the solution
assert canJumpBrute([2,3,1,1,4]) == True
assert canJumpBrute([3,2,1,0,4]) == False
assert canJumpBrute([2,0,2,0,0]) == True

In [6]:
# https://leetcode.com/problems/jump-game/submissions/1281092917
def canJump(nums: list[int]) -> bool:
    """
    We can notice that if the latest good index was reachable, then every other good index beyond would
    have been reachable as well.

    On the contrary if the latest good index (nearest to current) was not reachable, then no index beyond would be reachable.

    Time: O(N), Space: O(1)
    """
    N = len(nums)
    good_idx: int = N - 1
    for i in range(N - 2, -1, -1):
        if i + nums[i] >= good_idx:
            good_idx = i

    return good_idx == 0

# Testing the solution
assert canJump([2,3,1,1,4]) == True
assert canJump([3,2,1,0,4]) == False
assert canJump([2,0,2,0,0]) == True

In [7]:
# https://leetcode.com/problems/jump-game/submissions/1281113037
def canJumpStriver(nums: list[int]) -> bool:
    """
    If there were no 0's we would always be able to reach the destination.
    In other cases it may or may not be possible.

    Time: O(N), Space: O(1)
    """
    N, max_idx = len(nums), 0
    for i in range(N):
        if i <= max_idx:
            max_idx = max(max_idx, nums[i] + i)
        else:
            return False

    return True

# Testing the solution
assert canJumpStriver([2,3,1,1,4]) == True
assert canJumpStriver([3,2,1,0,4]) == False
assert canJumpStriver([2,0,2,0,0]) == True

Jump Game - II: https://leetcode.com/problems/jump-game-ii/
Video Link: https://youtu.be/7SBVnw7GSTk?si=UDZojaM70TuG0XgY

In [8]:
def jump2Recursive(nums: list[int]) -> int:
    @functools.cache
    def backtrack(i: int) -> int:
        if i == N - 1:
            return 0
        else:
            min_ = N
            for jump in range(nums[i]):
                min_ = min(min_, backtrack(i + jump + 1))
            return 1 + min_

    N = len(nums)
    return backtrack(0)

# Testing the solution
assert jump2Recursive([2,3,1,1,4]) == 2
assert jump2Recursive([2,3,0,1,4]) == 2

In [9]:
# https://leetcode.com/problems/jump-game-ii/submissions/1281812619/
def jump2Tab(nums: list[int]) -> int:
    # Time: O(N ** 2), Space: O(N)
    N = len(nums)
    dp: list[int] = [N if i < N - 1 else 0 for i in range(N)]
    for i in range(N - 2, -1, -1):
        for j in range(i + 1, min(N, i + 1 + nums[i])):
            dp[i] = min(dp[i], 1 + dp[j])

    return dp[0]

# Testing the solution
assert jump2Tab([2,3,1,1,4]) == 2
assert jump2Tab([2,3,0,1,4]) == 2

In [10]:
# https://leetcode.com/problems/jump-game-ii/submissions/1281839417
def jump2Striver(nums: list[int]) -> int:
    """
    Instead of iterating through all indices that we can jump to,
    we keep track of the range.
    For each range
        - Figure out the maximum idx we can jump to
        - jumps = jumps + 1
    Iterate until we reach the last index, ans would be the value of jumps variable

    Time: O(N), Space: O(1)
    """
    N = len(nums)
    jumps = l = r = 0
    while r < N - 1:
        farthest = 0
        for i in range(l, r + 1):
            farthest = max(farthest, i + nums[i])
        l, r, jumps = r + 1, farthest, jumps + 1

    return jumps

# Testing the solution
assert jump2Striver([2,3,1,1,4]) == 2
assert jump2Striver([2,3,0,1,4]) == 2

Job sequencing Problem: https://www.geeksforgeeks.org/problems/job-sequencing-problem-1587115620/1
Video Link: https://youtu.be/QbwltemZbRg?si=R93V-44Jyr37Il4L

In [11]:
def jobScheduling(jobs: list[tuple[int, int, int]], N: int) -> tuple[int, int]:
    """
    Start with the jobs with highest profit, try to push the start time as close as possible to the deadline
    being super lazy about it.

    Time: O(N log N) + O(N x max deadline)
    Space: O(N)

    We can optimize the inner loop (deadline -> 0) further with the use of DSU (Disjoint set union)
    """
    jobs_start: set[int] = set()
    jobs_sorted: list[tuple[int, int]] = sorted(map(lambda x: (x[2], x[1]), jobs), reverse=True)
    job_count = total_profit = 0
    for profit, deadline in jobs_sorted:
        for start in range(deadline - 1, -1, -1):
            if start not in jobs_start:
                jobs_start.add(start)
                job_count, total_profit = job_count + 1, total_profit + profit
                break

    return job_count, total_profit

# Testing the solution
assert jobScheduling([(1,2,100),(2,1,19),(3,2,27),(4,1,25),(5,1,15)], 5) == (2, 127)
assert jobScheduling([(1,4,20),(2,1,10),(3,1,40),(4,1,30)], 5) == (2, 60)
assert jobScheduling([(1,4,50),(2,1,50)], 2) == (2, 100)
assert jobScheduling([(1,4,60),(2,1,50)], 2) == (2, 110)

N meetings in one room: https://www.geeksforgeeks.org/problems/n-meetings-in-one-room-1587115620/1
Video Link: https://youtu.be/mKfhTotEguk?si=gTVT-RsSg0PR9RPP

In [12]:
def maximumMeetings(N: int, start: list[int], end: list[int]) -> int:
    """
    Sort based on end time, we need to find meetings that finish sooner.
    Simply iterate through the sorted meeting in order and check if it is possible to host the meeting

    - It works because the earliest-ending meeting can't possibly cause more conflicts than any other choice
    - It leaves the largest amount of un-interrupted time remaining for other intervals to be chosen from

    Time: O(N log N)
    Space: O(N)
    """

    # Sort meeting based on the end time
    meetings: list[tuple[int, int, int]] = [(start[i], end[i], i) for i in range(N)]
    meetings.sort(key=lambda x: x[1])

    order: list[int] = []
    meeting_count = free_at = 0
    for stime, etime, idx in meetings:
        if stime > free_at:
            order.append(idx)
            meeting_count, free_at = meeting_count + 1, etime

    # If needed, to return the order in which the meetings could be conducted,
    # we can make use of the `meeting_count` variable

    return meeting_count

# Testing the solution
assert maximumMeetings(6, [1,3,0,5,8,5], [2,4,6,7,9,9]) == 4
assert maximumMeetings(3, [1,4,6], [5,6,10]) == 2

Non overlapping intervals: https://leetcode.com/problems/non-overlapping-intervals/
Video Link: https://youtu.be/HDHQ8lAWakY?si=s1_2CwOoEBx5LSOQ

In [13]:
# https://leetcode.com/problems/non-overlapping-intervals/submissions/1282204448
def eraseOverlapIntervals(intervals: list[list[int]]) -> int:
    # Sort intervals based on start time
    intervals.sort(key=lambda x: (x[1], x[0]))

    print(intervals)

    curr_time, skip = intervals[0][0], 0
    for stime, etime in intervals:
        if curr_time <= stime:
            curr_time = etime
        else:
            skip += 1

    return skip

# Testing the solution
assert eraseOverlapIntervals([[1,2],[2,3],[3,4],[1,3]]) == 1
assert eraseOverlapIntervals([[1,2],[1,2],[1,2]]) == 2
assert eraseOverlapIntervals([[1,2],[2,3]]) == 0
assert eraseOverlapIntervals([[-10,20],[8,12],[3,4],[5,6],[10,12],[15,20]]) == 2

[[1, 2], [1, 3], [2, 3], [3, 4]]
[[1, 2], [1, 2], [1, 2]]
[[1, 2], [2, 3]]
[[3, 4], [5, 6], [8, 12], [10, 12], [-10, 20], [15, 20]]


Insert intervals: https://leetcode.com/problems/insert-interval/description/
Video Link: https://youtu.be/xxRE-46OCC8?si=kKQ10RKueMFdlR-e

In [14]:
# https://leetcode.com/problems/insert-interval/submissions/1282223708/
def insertIntervals(intervals: list[list[int]], newInterval: list[int]) -> list[list[int]]:
    # Time: O(N), Space: O(N)
    N = len(intervals)

    # Insert intervals here post merge
    merged: list[list[int]] = []

    # Travel until we are at a pos to insert newInterval
    i = 0
    while i < N and intervals[i][0] < newInterval[0]:
        merged.append(intervals[i])
        i += 1

    # Insert the new interval or merge it with prev interval
    if merged and merged[-1][1] >= newInterval[0]:
        merged[-1] = [merged[-1][0], max(merged[-1][1], newInterval[1])]
    else:
        merged.append(newInterval)

    # Continue merging and then inserting until all intervals are done
    while i < N:
        if merged[-1][1] >= intervals[i][0]:
            merged[-1] = [merged[-1][0], max(merged[-1][1], intervals[i][1])]
        else:
            merged.append(intervals[i])
        i += 1

    return merged

# Testing the solution
assert insertIntervals([[1,3],[6,9]], [2,5]) == [[1,5],[6,9]]
assert insertIntervals([[1,2],[3,5],[6,7],[8,10],[12,16]], [3,8]) == [[1,2],[3,10],[12,16]]
assert insertIntervals([[1,2],[3,5],[6,7],[8,10],[12,16]], [11,11]) == [[1,2],[3,5],[6,7],[8,10],[11,11],[12,16]]

Minimum number of platforms required in a railway station: https://www.geeksforgeeks.org/problems/minimum-platforms-1587115620/1
Video link: https://youtu.be/AsGzwR_FWok?si=Nt5CMUPctV39JtSd

In [15]:
def minimumPlatformBrute1(N: int, arr: list[int], dep: list[int]) -> int:
    # Time: O(N x 2400) + O(2400), Space: O(2400)
    platform: list[int] = [0 for i in range(2400)]
    for stime, etime in zip(arr, dep):
        for i in range(stime, etime + 1):
            platform[i] += 1

    return max(platform)

# Testing the soltution
assert minimumPlatformBrute1(3, [900, 1100, 1235], [1000, 1200, 1240]) == 1
assert minimumPlatformBrute1(3, [900, 930, 1015], [1000, 1030, 1100]) == 2
assert minimumPlatformBrute1(3, [900, 930, 1000], [1000, 1030, 1100]) == 3

In [16]:
def minimumPlatformBrute2(N: int, arr: list[int], dep: list[int]) -> int:
    max_overlap = 0
    for i in range(N):
        overlap, istart, iend = 0, arr[i], dep[i]
        for j in range(i + 1, N):
            jstart, jend = arr[j], dep[j]
            if jstart <= iend:
                overlap += 1
        max_overlap = max(max_overlap, overlap)

    return max_overlap + 1

# Testing the soltution
assert minimumPlatformBrute2(3, [900, 1100, 1235], [1000, 1200, 1240]) == 1
assert minimumPlatformBrute2(3, [900, 930, 1015], [1000, 1030, 1100]) == 2
assert minimumPlatformBrute2(3, [900, 930, 1000], [1000, 1030, 1100]) == 3

In [17]:
def minimumPlatform(N: int, arr: list[int], dep: list[int]) -> int:
    """
    Brilliant logic from Striver:
        Stand and observe the station as the time passes by

    Time: O(N log N), Space: O(1)
    """
    # Sort the events based on time, imagine an observer standing by and watching the events unfolding
    arr.sort()
    dep.sort()

    # Count the overlaps
    max_overlap = overlap = arr_idx = dep_idx = 0
    while arr_idx < N or dep_idx < N:
        if dep_idx >= N or (arr_idx < N and arr[arr_idx] <= dep[dep_idx]):
            overlap, arr_idx = overlap + 1, arr_idx + 1
        else:
            overlap, dep_idx = overlap - 1, dep_idx + 1
        max_overlap = max(max_overlap, overlap)

    return max_overlap

# Testing the soltution
assert minimumPlatform(3, [900, 1100, 1235], [1000, 1200, 1240]) == 1
assert minimumPlatform(3, [900, 930, 1015], [1000, 1030, 1100]) == 2
assert minimumPlatform(3, [900, 930, 1000], [1000, 1030, 1100]) == 3

Valid Parenthesis String: https://leetcode.com/problems/valid-parenthesis-string/
Video Link: https://youtu.be/cHT6sG_hUZI?si=7S_ZYJK-16nXDsLR

In [ ]:
def checkValidStringBrute(s: str) -> bool:
    @functools.cache
    def backtrack(i: int, brackets: int) -> bool:
        if brackets < 0:
            return False
        elif i == N:
            return brackets == 0
        elif s[i] == "(":
            return backtrack(i + 1, brackets + 1)
        elif s[i] == ")":
            return backtrack(i + 1, brackets - 1)
        else:
            return backtrack(i + 1, brackets) or backtrack(i + 1, brackets - 1) or backtrack(i + 1, brackets + 1)

    N = len(s)
    return backtrack(0, 0)

# Testing the solution
for s, result in [
        ("()", True), ("(*)", True), ("(*))", True), ("(*()", True), (")*", False),
        ("(**", True), ("*", True), ("(**)()(((*))*))", True), ("(**(", False)
    ]:
    assert checkValidStringBrute(s) == result, f"failed for {s}"

In [ ]:
def checkValidStringBetter(s: str) -> bool:
    """
    Tabulating the backtracking approach to bring down log(N) portion of runtime

    Time: O(N ** 2), Space: O(N ** 2)
    """

    N = len(s)
    dp: list[list[bool]] = [[True if i == N and j == 0 else False for j in range(N + 1)] for i in range(N + 1)]
    for i in range(N - 1, -1, -1):
        for j in range(N - 1, -1, -1):
            if s[i] == "(":
                dp[i][j] = dp[i + 1][j + 1]
            elif s[i] == ")":
                dp[i][j] = dp[i + 1][j - 1]
            else:
                dp[i][j] = dp[i + 1][j] or dp[i + 1][j - 1] or dp[i + 1][j + 1]

    return dp[0][0]

# Testing the solution
for s, result in [
        ("()", True), ("(*)", True), ("(*))", True), ("(*()", True), (")*", False),
        ("(**", True), ("*", True), ("(**)()(((*))*))", True), ("(**(", False)
    ]:
    assert checkValidStringBetter(s) == result, f"failed for {s}"

In [ ]:
# https://leetcode.com/problems/valid-parenthesis-string/submissions/1282562888/
def checkValidStringSpaceOptimized(s: str) -> bool:
    """
    Time: O(N ** 2), Space: O(N)
    """
    N = len(s)
    dp: list[bool] = [True if j == 0 else False for j in range(N + 1)]
    for i in range(N - 1, -1, -1):
        prev_dp: list[bool] = list(dp)
        for j in range(N - 1, -1, -1):
            if s[i] == "(":
                dp[j] = prev_dp[j + 1]
            elif s[i] == ")":
                dp[j] = prev_dp[j - 1]
            else:
                dp[j] = prev_dp[j] or prev_dp[j - 1] or prev_dp[j + 1]

    return dp[0]

# Testing the solution
for s, result in [
        ("()", True), ("(*)", True), ("(*))", True), ("(*()", True), (")*", False),
        ("(**", True), ("*", True), ("(**)()(((*))*))", True), ("(**(", False)
    ]:
    assert checkValidStringSpaceOptimized(s) == result, f"failed for {s}"

In [ ]:
# https://leetcode.com/problems/valid-parenthesis-string/submissions/1282590335
def checkValidString(s: str) -> bool:
    """
    We still try out all possibilties, instead of carrying a count of brackets we carry
    a range.

    Time: O(N), Space: O(1)
    """
    low = high = 0
    for ch in s:
        if ch == "(":
            low, high = low + 1, high + 1
        elif ch == ")":
            low, high = low - 1, high - 1
        else:
            low, high = low - 1, high + 1

        if low < 0:
            low = 0
        if high < 0:
            return False

    return low == 0

# Testing the solution
for s, result in [
        ("()", True), ("(*)", True), ("(*))", True), ("(*()", True), (")*", False),
        ("(**", True), ("*", True), ("(**)()(((*))*))", True), ("(**(", False)
    ]:
    assert checkValidString(s) == result, f"failed for {s}"

Fractional Knapsack: https://www.geeksforgeeks.org/problems/fractional-knapsack-1587115620/1
Video Link: https://youtu.be/1ibsQrnuEEg?si=nTPVv4QkGEXUTndS

In [ ]:
def fractionalknapsack(capacity: int, values: list[float], weights: list[int], N: int) -> float:
    """
    Time: O(N log N), Space: O(N)
    """
    # We use the unit rate to pick the best possible item to pick
    rates: list[tuple[float, int]] = []
    for i in range(N):
        rates.append((values[i] / weights[i], weights[i]))
    rates.sort(reverse=True)

    # Greedily fill in as much as possible
    total = 0.
    idx = 0
    while idx < N:
        units = min(capacity, rates[idx][1])
        total += rates[idx][0] * units
        capacity -= units
        idx += 1
        if capacity == 0:
            break

    return total

# Testing the solution
assert fractionalknapsack(50, [60,100,120], [10,20,30], 3) == 240.
assert fractionalknapsack(50, [60,100], [10,20], 2) == 160.

Candy: https://leetcode.com/problems/candy/
Video link: https://youtu.be/IIqVFvKE6RY?si=tMFSEL4aw1g2VSc5

In [ ]:
def candy(ratings: list[int]) -> int:
    """
    Start with kids having lowest ratings. Assign them 1 candy and try to move left/right.

    If neighbouring kid has a higher rating, we need to give them atleast 1 candy more than what current kid has.
    If neighbouring kid has a lower rating, we could assign them the lowest possible as long as they are isolated* (check if they are untouched previously)

    Time: O(N log N), Space: O(N)
    """
    N, min_rating = len(ratings), min(ratings)
    candies: list[int] = [-1 for i in range(N)]
    heap: list[tuple[int, int]] = []
    for i in range(N):
        if ratings[i] == min_rating:
            heap.append((1, i))

    while heap:
        count, idx = heapq.heappop(heap)
        candies[idx] = count

        if idx - 1 >= 0:
            # Neighbour has better rating and lower number of candies, update ->
            if ratings[idx - 1] > ratings[idx] and candies[idx - 1] <= candies[idx]:
                heapq.heappush(heap, (count + 1, idx - 1))

            # Try to sneakily assign the lowest possible candy, if they are isolated (untouched previously)
            elif candies[idx - 1] == -1:
                heapq.heappush(heap, (1, idx - 1))

        if idx + 1 < N:
            # Neighbour has better rating and lower number of candies, update ->
            if ratings[idx + 1] > ratings[idx] and candies[idx + 1] <= candies[idx]:
                heapq.heappush(heap, (count + 1, idx + 1))

            # Try to sneakily assign the lowest possible candy, if they are isolated (untouched previously)
            elif candies[idx + 1] == -1:
                heapq.heappush(heap, (1, idx + 1))

    return sum(candies)

# Testing the solution
assert candy([1,0,2]) == 5
assert candy([1,2,2]) == 4
assert candy([3,2,1]) == 6
assert candy([1,3,4,5,2]) == 11

In [ ]:
# https://leetcode.com/problems/candy/submissions/1282748617
def candyStriverBrute(ratings: list[int]) -> int:
    # Time: O(N), Space: O(N)
    N = len(ratings)

    # Solve worrying only about the left neighbours
    left: list[int] = []
    for i in range(N):
        if i == 0 or ratings[i] <= ratings[i - 1]:
            left.append(1)
        else:
            left.append(left[-1] + 1)

    # Solve worrying only about the right neighbours
    right: list[int] = []
    for i in range(N - 1, -1, -1):
        if i == N - 1 or ratings[i] <= ratings[i + 1]:
            right.append(1)
        else:
            right.append(right[-1] + 1)
    right.reverse()

    # The candies assigned must satisfy both left and right, hence max
    total = 0
    for i in range(N):
        total += max(left[i], right[i])
    return total

# Testing the solution
assert candyStriverBrute([1,0,2]) == 5
assert candyStriverBrute([1,2,2]) == 4
assert candyStriverBrute([3,2,1]) == 6
assert candyStriverBrute([1,3,4,5,2]) == 11

In [ ]:
# https://leetcode.com/problems/candy/submissions/1282800027/
def candyStriver(ratings: list[int]) -> int:
    """
    Won't be able to think this solution on our own, need some prior insights
    Visualize ratings in terms of slope - peaks and troughs

    Code shamelessly copied from Striver's logic :O
    """
    N = len(ratings)
    total = i = 1
    while i < N:
        if i == 0 or ratings[i] == ratings[i - 1]:
            total, i = total + 1, i + 1
        else:
            peak = 1
            while i < N and ratings[i] > ratings[i - 1]:
                peak += 1
                total, i = total + peak, i + 1

            down = 1
            while i < N and ratings[i] < ratings[i - 1]:
                total += down
                down, i = down + 1, i + 1

            if down > peak:
                total += down - peak

    return total

# Testing the solution
assert candyStriver([1,0,2]) == 5
assert candyStriver([1,2,2]) == 4
assert candyStriver([3,2,1]) == 6
assert candyStriver([1,3,4,5,2]) == 11